In [1]:
import pandas as pd
import pymongo
import json
import datetime as dt
import calendar

In [2]:
# import all cvs from data folder
earthquake_csv = pd.read_csv("../raw/Earthquake_data.csv")
floods_csv = pd.read_csv("../floods_data.csv")
tornado_csv = pd.read_csv("../tor_output.csv")
hurricane_csv = pd.read_csv("../hurricane_cleaned_df.csv")

C:\Users\David\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
earthquake_df = pd.DataFrame(earthquake_csv)
floods_df = pd.DataFrame(floods_csv)
hurricane_df = pd.DataFrame(hurricane_csv)
tornado_df = pd.DataFrame(tornado_csv)

## EARTHQUAKE DATA CLEANING

In [4]:
earthquake_df["Date"] = earthquake_df.Year.map(str)+'-'+ earthquake_df.Month.map(str) + '-' + earthquake_df.Day.map(str)

In [5]:
earthquake_df.Date = pd.to_datetime(earthquake_df.Date)

In [6]:
earthquake_df = earthquake_df.fillna(0)

In [7]:
earthquake_df = earthquake_df.dropna()

In [8]:
earthquake_df.Hour = earthquake_df.Hour.astype(int)

In [9]:
earthquake_df.Minute = earthquake_df.Minute.astype(int)

In [10]:
earthquake_df.Minute = earthquake_df.Minute.map("{:02}".format)  
earthquake_df.Hour  = earthquake_df.Hour.map("{:02}".format)

In [11]:
earthquake_df["Time"] = earthquake_df.Hour.map(str) + ":" + earthquake_df.Minute

In [12]:
earthquake_df = earthquake_df.rename(columns = {"Latitude": "latitude", "Longitude": "longitude"})

In [13]:
earthquake_df.Time = pd.to_datetime(earthquake_df.Time).dt.time

In [14]:
earthquake_df['Month'] = earthquake_df['Month'].apply(lambda x: calendar.month_abbr[x])

In [15]:
earthquake_df

,Year,Month,Day,Hour,Minute,Location,latitude,longitude,Magnitude,Deaths,Injuries,Damage in Millions,Damage Scale,Number of Houses Destroyed,Scale of houses destroyed,Number of Houses Damaged,Scale of houses damaged,Date,Time
0,2017,May,1,12,31,ALASKA: SKAGWAY; CANADA: BRITISH COLUMBIA,59.852,-136.677,6.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-05-01,12:31:00
1,1988,Nov,25,23,46,"CANADA: QUEBEC: SAGUENAY, QUEBEC CITY",48.117,-71.183,5.8,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1988-11-25,23:46:00
2,2010,Jun,23,17,41,"CANADA: QUEBEC: VAL-DES-BOIS, GRACEFIELD",45.880,-75.480,5.2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2010-06-23,17:41:00
3,2001,Oct,12,05,02,CANADA: QUEEN CHARLOTTE ISLANDS,52.630,-132.200,6.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2001-10-12,05:02:00
4,2012,Oct,28,03,04,CANADA: QUEEN CHARLOTTE ISLANDS,52.788,-132.101,7.7,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2012-10-28,03:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,1980,May,18,15,32,WASHINGTON: MT ST HELENS,46.214,-122.194,5.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1980-05-18,15:32:00
190,2001,Feb,28,18,54,"WASHINGTON: OLYMPIA, SEATTLE, TACOMA",47.149,-122.727,6.8,1.0,400.0,2000.0,4.0,0.0,0.0,0.0,0.0,2001-02-28,18:54:00
191,1965,Apr,29,15,28,WASHINGTON: SEATTLE,47.400,-122.300,6.6,7.0,0.0,28.0,4.0,0.0,0.0,0.0,0.0,1965-04-29,15:28:00
192,1994,Feb,3,09,05,WYOMING: AFTON,42.762,-110.976,5.8,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1994-02-03,09:05:00


## HURRICANE DATA CLEANING

In [16]:
hurricane_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57095 entries, 0 to 57094
Data columns (total 12 columns):
Unnamed: 0      57095 non-null int64
index           57095 non-null int64
season          57095 non-null int64
basin           57095 non-null object
subbasin        57095 non-null object
name            57095 non-null object
iso_time        57095 non-null object
nature          57095 non-null object
latitude        57095 non-null float64
longitude       57095 non-null float64
usa_wind        57095 non-null float64
usa_pressure    30937 non-null float64
dtypes: float64(4), int64(3), object(5)
memory usage: 5.2+ MB


In [17]:
hurricane_df.head()

,Unnamed: 0,index,season,basin,subbasin,name,iso_time,nature,latitude,longitude,usa_wind,usa_pressure
0,0,1,2018,EP,MM,XAVIER,2018-11-06 06:00:00 UTC,NR,18.9371,-107.832,38.0,1005.0
1,1,2,2018,EP,CP,OLIVIA,2018-09-14 06:00:00 UTC,NR,18.9405,-164.639,29.0,1008.0
2,2,5,2018,EP,CP,OLIVIA,2018-09-14 12:00:00 UTC,NR,18.8559,-165.699,29.0,1008.0
3,3,11,1966,EP,CP,KATHY,1966-10-20 06:00:00 UTC,TS,44.6143,180.000,55.0,NaN
4,4,12,1992,EP,CP,DAN,1992-10-25 06:00:00 UTC,TS,11.6714,180.229,30.0,NaN


In [18]:
hurricane_df = hurricane_df.rename(columns = {"season": "Year"})

In [19]:
hurricane_df = hurricane_df.drop(columns = ["Unnamed: 0"])

In [20]:
hurricane_df[["Date","Time", "Time_Zone"]] = hurricane_df.iso_time.str.split(" ", expand = True)

In [21]:
hurricane_df[["Year", "Month","Day"]] = hurricane_df.Date.str.split("-", expand = True)

In [22]:
hurricane_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57095 entries, 0 to 57094
Data columns (total 16 columns):
index           57095 non-null int64
Year            57095 non-null object
basin           57095 non-null object
subbasin        57095 non-null object
name            57095 non-null object
iso_time        57095 non-null object
nature          57095 non-null object
latitude        57095 non-null float64
longitude       57095 non-null float64
usa_wind        57095 non-null float64
usa_pressure    30937 non-null float64
Date            57095 non-null object
Time            57095 non-null object
Time_Zone       57095 non-null object
Month           57095 non-null object
Day             57095 non-null object
dtypes: float64(4), int64(1), object(11)
memory usage: 7.0+ MB


In [23]:
hurricane_df.Month = hurricane_df.Month.astype(int)
hurricane_df.Year = hurricane_df.Year.astype(int)
hurricane_df.Day = hurricane_df.Day.astype(int)

In [24]:
hurricane_df['Month'] = hurricane_df['Month'].apply(lambda x: calendar.month_abbr[x])

In [25]:
hurricane_df.Date = pd.to_datetime(hurricane_df.Date)
hurricane_df.Time = pd.to_datetime(hurricane_df.Time).dt.time

## FLOOD DATA CLEANING

In [26]:
floods_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34586 entries, 0 to 34585
Data columns (total 17 columns):
episode_id          34586 non-null int64
state               34586 non-null object
event_type          34586 non-null object
event_begin_time    34586 non-null object
event_end_time      34586 non-null object
event_timezone      34586 non-null object
damage_property     34586 non-null float64
event_latitude      34586 non-null float64
event_longitude     34586 non-null float64
cz_type             34586 non-null object
cz_fips_code        34586 non-null int64
damage_crops        34586 non-null float64
source              34586 non-null object
Start_Date          34586 non-null object
Start_Time          34586 non-null object
End_Date            34586 non-null object
End_Time            34586 non-null object
dtypes: float64(4), int64(2), object(11)
memory usage: 4.5+ MB


In [27]:
floods_df[["Date", "Time"]] = floods_df.event_begin_time.str.split(" ", expand = True)

In [28]:
floods_df[["Year", "Month", "Day"]] = floods_df.Start_Date.str.split("-", expand = True)

In [29]:
floods_df.Month = floods_df.Month.astype(int)

In [30]:
floods_df['Month'] = floods_df['Month'].apply(lambda x: calendar.month_abbr[x])

In [31]:
floods_df.Date = pd.to_datetime(floods_df.Date)
floods_df.Time = pd.to_datetime(floods_df.Time).dt.time

In [32]:
floods_df = floods_df.rename(columns = {"event_latitude": "latitude", "event_longitude": "longitude"})


In [33]:
floods_df = floods_df.drop(columns = ["Start_Date", "End_Time", "End_Date","End_Date","Start_Time"])

In [34]:
floods_df.event_begin_time = pd.to_datetime(floods_df.event_begin_time)
floods_df.event_end_time = pd.to_datetime(floods_df.event_end_time)


In [35]:
floods_df.Year = floods_df.Year.astype(int)

## TORNADO DATA CLEANING

In [36]:
tornado_df.head()

,Date,Time,State,Fscale,Injured,Fatalities,Property_loss,SLAT,SLON,ELAT,ELON,Length,Width
0,1950-01-03,11:00:00,MO,3,3.0,0.0,6.0,38.77,-90.22,38.83,-90.03,15.2855,137.111517
1,1950-01-03,11:55:00,IL,3,3.0,0.0,5.0,39.10,-89.30,39.12,-89.23,5.7924,118.829982
2,1950-01-03,16:00:00,OH,1,1.0,0.0,4.0,40.88,-84.58,0.00,0.00,0.1609,9.140768
3,1950-01-13,05:25:00,AR,3,1.0,1.0,3.0,34.40,-94.37,0.00,0.00,0.9654,15.539305
4,1950-01-25,19:30:00,MO,2,5.0,0.0,5.0,37.60,-90.68,37.63,-90.65,3.7007,274.223035


In [37]:
tornado_df[["Year", "Month", "Day"]] = tornado_df.Date.str.split("-", expand = True)


In [38]:
tornado_df.Month = tornado_df.Month.astype(int)

In [39]:
tornado_df['Month'] = tornado_df['Month'].apply(lambda x: calendar.month_abbr[x])

In [40]:
tornado_df = tornado_df.drop(columns = ["Day"])

In [41]:
tornado_df.Date = pd.to_datetime(tornado_df.Date, errors = 'coerce')
# tornado_df.Time = pd.to_datetime(tornado_df.Time, errors = 'coerce')


In [42]:
tornado_df.Property_loss = tornado_df.Property_loss.replace({'K': '*1e3', 'M': '*1e6'}, regex=True).map(pd.eval).astype(int)

In [43]:
tornado_df=tornado_df.dropna()

In [44]:
tornado_df.Year = tornado_df.Year.astype(int)

## Change all column to lower case

In [45]:
floods_df.columns = map(str.lower, floods_df.columns)
earthquake_df.columns = map(str.lower, earthquake_df.columns)
hurricane_df.columns = map(str.lower, hurricane_df.columns)
tornado_df.columns = map(str.lower, tornado_df.columns)

## View all DataFrames

In [46]:
floods_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34586 entries, 0 to 34585
Data columns (total 18 columns):
episode_id          34586 non-null int64
state               34586 non-null object
event_type          34586 non-null object
event_begin_time    34586 non-null datetime64[ns]
event_end_time      34586 non-null datetime64[ns]
event_timezone      34586 non-null object
damage_property     34586 non-null float64
latitude            34586 non-null float64
longitude           34586 non-null float64
cz_type             34586 non-null object
cz_fips_code        34586 non-null int64
damage_crops        34586 non-null float64
source              34586 non-null object
date                34586 non-null datetime64[ns]
time                34586 non-null object
year                34586 non-null int32
month               34586 non-null object
day                 34586 non-null object
dtypes: datetime64[ns](3), float64(4), int32(1), int64(2), object(8)
memory usage: 4.6+ MB


In [47]:
hurricane_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57095 entries, 0 to 57094
Data columns (total 16 columns):
index           57095 non-null int64
year            57095 non-null int32
basin           57095 non-null object
subbasin        57095 non-null object
name            57095 non-null object
iso_time        57095 non-null object
nature          57095 non-null object
latitude        57095 non-null float64
longitude       57095 non-null float64
usa_wind        57095 non-null float64
usa_pressure    30937 non-null float64
date            57095 non-null datetime64[ns]
time            57095 non-null object
time_zone       57095 non-null object
month           57095 non-null object
day             57095 non-null int32
dtypes: datetime64[ns](1), float64(4), int32(2), int64(1), object(8)
memory usage: 6.5+ MB


In [48]:
tornado_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66136 entries, 0 to 66140
Data columns (total 15 columns):
date             66136 non-null datetime64[ns]
time             66136 non-null object
state            66136 non-null object
fscale           66136 non-null object
injured          66136 non-null float64
fatalities       66136 non-null float64
property_loss    66136 non-null int32
slat             66136 non-null float64
slon             66136 non-null float64
elat             66136 non-null float64
elon             66136 non-null float64
length           66136 non-null float64
width            66136 non-null float64
year             66136 non-null int32
month            66136 non-null object
dtypes: datetime64[ns](1), float64(8), int32(2), object(4)
memory usage: 7.6+ MB


## Convert all the dataframes into json format

In [49]:
hurricane_data = json.loads(hurricane_df.T.to_json()).values()
floods_data = json.loads(floods_df.T.to_json()).values()
earthquake_data = json.loads(earthquake_df.T.to_json()).values()
tornado_data = json.loads(tornado_df.T.to_json()).values()

## NoSQL

In [50]:
# connect to mongod local server
conn = 'mongodb://localhost:27017'
# create client 
client = pymongo.MongoClient(conn)
# create database
# db.weather_data.drop()
db = client.weather_data


In [51]:
db.hurricane_data.drop()
db.hurricane_data.insert(hurricane_data)

C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


[ObjectId('5e22c5a86106ad075407d951'),
 ObjectId('5e22c5a86106ad075407d952'),
 ObjectId('5e22c5a86106ad075407d953'),
 ObjectId('5e22c5a86106ad075407d954'),
 ObjectId('5e22c5a86106ad075407d955'),
 ObjectId('5e22c5a86106ad075407d956'),
 ObjectId('5e22c5a86106ad075407d957'),
 ObjectId('5e22c5a86106ad075407d958'),
 ObjectId('5e22c5a86106ad075407d959'),
 ObjectId('5e22c5a86106ad075407d95a'),
 ObjectId('5e22c5a86106ad075407d95b'),
 ObjectId('5e22c5a86106ad075407d95c'),
 ObjectId('5e22c5a86106ad075407d95d'),
 ObjectId('5e22c5a86106ad075407d95e'),
 ObjectId('5e22c5a86106ad075407d95f'),
 ObjectId('5e22c5a86106ad075407d960'),
 ObjectId('5e22c5a86106ad075407d961'),
 ObjectId('5e22c5a86106ad075407d962'),
 ObjectId('5e22c5a86106ad075407d963'),
 ObjectId('5e22c5a86106ad075407d964'),
 ObjectId('5e22c5a86106ad075407d965'),
 ObjectId('5e22c5a86106ad075407d966'),
 ObjectId('5e22c5a86106ad075407d967'),
 ObjectId('5e22c5a86106ad075407d968'),
 ObjectId('5e22c5a86106ad075407d969'),
 ObjectId('5e22c5a86106ad

In [52]:
db.flood_data.drop()
db.flood_data.insert(floods_data)

C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


[ObjectId('5e22c5a96106ad075408b858'),
 ObjectId('5e22c5a96106ad075408b859'),
 ObjectId('5e22c5a96106ad075408b85a'),
 ObjectId('5e22c5a96106ad075408b85b'),
 ObjectId('5e22c5a96106ad075408b85c'),
 ObjectId('5e22c5a96106ad075408b85d'),
 ObjectId('5e22c5a96106ad075408b85e'),
 ObjectId('5e22c5a96106ad075408b85f'),
 ObjectId('5e22c5a96106ad075408b860'),
 ObjectId('5e22c5a96106ad075408b861'),
 ObjectId('5e22c5a96106ad075408b862'),
 ObjectId('5e22c5a96106ad075408b863'),
 ObjectId('5e22c5a96106ad075408b864'),
 ObjectId('5e22c5a96106ad075408b865'),
 ObjectId('5e22c5a96106ad075408b866'),
 ObjectId('5e22c5a96106ad075408b867'),
 ObjectId('5e22c5a96106ad075408b868'),
 ObjectId('5e22c5a96106ad075408b869'),
 ObjectId('5e22c5a96106ad075408b86a'),
 ObjectId('5e22c5a96106ad075408b86b'),
 ObjectId('5e22c5a96106ad075408b86c'),
 ObjectId('5e22c5a96106ad075408b86d'),
 ObjectId('5e22c5a96106ad075408b86e'),
 ObjectId('5e22c5a96106ad075408b86f'),
 ObjectId('5e22c5a96106ad075408b870'),
 ObjectId('5e22c5a96106ad

In [53]:
db.earthquake_data.drop()
db.earthquake_data.insert(earthquake_data)

C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


[ObjectId('5e22c5aa6106ad0754093f72'),
 ObjectId('5e22c5aa6106ad0754093f73'),
 ObjectId('5e22c5aa6106ad0754093f74'),
 ObjectId('5e22c5aa6106ad0754093f75'),
 ObjectId('5e22c5aa6106ad0754093f76'),
 ObjectId('5e22c5aa6106ad0754093f77'),
 ObjectId('5e22c5aa6106ad0754093f78'),
 ObjectId('5e22c5aa6106ad0754093f79'),
 ObjectId('5e22c5aa6106ad0754093f7a'),
 ObjectId('5e22c5aa6106ad0754093f7b'),
 ObjectId('5e22c5aa6106ad0754093f7c'),
 ObjectId('5e22c5aa6106ad0754093f7d'),
 ObjectId('5e22c5aa6106ad0754093f7e'),
 ObjectId('5e22c5aa6106ad0754093f7f'),
 ObjectId('5e22c5aa6106ad0754093f80'),
 ObjectId('5e22c5aa6106ad0754093f81'),
 ObjectId('5e22c5aa6106ad0754093f82'),
 ObjectId('5e22c5aa6106ad0754093f83'),
 ObjectId('5e22c5aa6106ad0754093f84'),
 ObjectId('5e22c5aa6106ad0754093f85'),
 ObjectId('5e22c5aa6106ad0754093f86'),
 ObjectId('5e22c5aa6106ad0754093f87'),
 ObjectId('5e22c5aa6106ad0754093f88'),
 ObjectId('5e22c5aa6106ad0754093f89'),
 ObjectId('5e22c5aa6106ad0754093f8a'),
 ObjectId('5e22c5aa6106ad

In [54]:
db.tornado_data.drop()
db.tornado_data.insert(tornado_data)

C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


[ObjectId('5e22c5aa6106ad0754094034'),
 ObjectId('5e22c5aa6106ad0754094035'),
 ObjectId('5e22c5aa6106ad0754094036'),
 ObjectId('5e22c5aa6106ad0754094037'),
 ObjectId('5e22c5aa6106ad0754094038'),
 ObjectId('5e22c5aa6106ad0754094039'),
 ObjectId('5e22c5aa6106ad075409403a'),
 ObjectId('5e22c5aa6106ad075409403b'),
 ObjectId('5e22c5aa6106ad075409403c'),
 ObjectId('5e22c5aa6106ad075409403d'),
 ObjectId('5e22c5aa6106ad075409403e'),
 ObjectId('5e22c5aa6106ad075409403f'),
 ObjectId('5e22c5aa6106ad0754094040'),
 ObjectId('5e22c5aa6106ad0754094041'),
 ObjectId('5e22c5aa6106ad0754094042'),
 ObjectId('5e22c5aa6106ad0754094043'),
 ObjectId('5e22c5aa6106ad0754094044'),
 ObjectId('5e22c5aa6106ad0754094045'),
 ObjectId('5e22c5aa6106ad0754094046'),
 ObjectId('5e22c5aa6106ad0754094047'),
 ObjectId('5e22c5aa6106ad0754094048'),
 ObjectId('5e22c5aa6106ad0754094049'),
 ObjectId('5e22c5aa6106ad075409404a'),
 ObjectId('5e22c5aa6106ad075409404b'),
 ObjectId('5e22c5aa6106ad075409404c'),
 ObjectId('5e22c5aa6106ad